In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from pymoo.factory import get_performance_indicator
from itertools import product

pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format',lambda x : '%.4f' % x)
plt.style.use(['science','ieee'])


In [ ]:
results_list1 = [
    "results-database-20220126085759.json"
]

from tinydb import TinyDB,Query
dataset1=pd.DataFrame()

for results in results_list1:
    database = TinyDB("results/"+results)
    q=Query()
    temp=pd.DataFrame(database.all())
    dataset1=pd.concat([dataset1,temp])
dataset1 = dataset1[["run", "s", "system_accuracies","points","constraints","experiment_time"]]
dataset1=dataset1.fillna(0)
dataset1[['run','s']] = dataset1[['run', 's']].astype(int)
refs1 = pd.DataFrame()
refs1['accuracy'] = dataset1['constraints'].map(lambda x: x['accuracy'][0])
refs1['number'] = dataset1['constraints'].map(lambda x: x['number'][0])
refs1['bandwidth'] = dataset1['constraints'].map(lambda x: x['bandwidth'][0])
ref1 = refs1.min().tolist()
dataset1 = dataset1.drop(columns=["constraints"])

hv1 = get_performance_indicator("hv", ref_point=np.array(ref1) * (-1))


In [ ]:
time_results1 = dataset1[['s','run','experiment_time']]
time_results1 = time_results1.drop(columns=["run"])
time_results1=time_results1.rename(columns={"experiment_time": "Time"})
time_results1


In [ ]:
df1=dataset1.copy(deep=True)
heuristic_dataset1=[]
if len(df1.index)>0:
    df1 = df1[['points', 'run', 's']]
    df1=list(df1.to_dict('records'))

    for item in df1:
        item['policy']="S"+str(item['s'])
        points=item.pop('points')
        for point in points:
            point.update(item)
            heuristic_dataset1.append(point)

df1=pd.DataFrame(heuristic_dataset1).drop_duplicates(subset=None, keep='first', inplace=False)
# df.head()
hv_results1 = []
for s, run in product(df1['s'].unique(), df1['run'].unique()):
    filter1 = df1['s'] == s
    filter2 = df1['run'] == run
    temp = df1[['served', 'average_accuracy', 'bandwidth_reserved', 's',
               'run']].where(filter1 & filter2).dropna(axis=0, how='any')
    if temp.size > 0:
        data = temp[['average_accuracy', 'served', 'bandwidth_reserved']]
        result = {'s': s, 'run': run}
        front = data.values
        result.update({"HV": hv1.do(-front)})
        hv_results1.append(result)
hv_results1 = pd.DataFrame(hv_results1)
hv_results1

In [ ]:
plt.rcParams["axes.formatter.useoffset"] = True
from palettable.scientific.diverging import Vik_7

hv_results1[['s']] = hv_results1[['s']].astype("str")
time_results1[['s']] = time_results1[['s']].astype("str")

fig, ax1 = plt.subplots(figsize=(4, 2))

sns.lineplot(x='s',
             y='HV',
             data=hv_results1,
             marker='o',
             markeredgecolor=None,
             markeredgewidth=None,
             markersize=4,
             linestyle='-',
             color=Vik_7.mpl_colors[1],
             ax=ax1)

ax2 = ax1.twinx()
ax2 = sns.lineplot(x='s',
                   y='Time',
                   data=time_results1,
                   color=Vik_7.mpl_colors[5],
                   ax=ax2,
                   marker='X',
                   markeredgecolor=None,
                   markeredgewidth=None,
                   markersize=4,
                   linestyle='-')

ax1.xaxis.label.set_text('\\textbf{\# of Weight Vectors ($\\times 1000$)}')
ax2.yaxis.label.set_text('\\textbf{ TC (sec.)}')
ax1.yaxis.label.set_text('\\textbf{HV}')
ax1.set_ylim(990,1080)
ax1.yaxis.set_tick_params(labelsize='small')
ax2.yaxis.set_tick_params(labelsize='small')
plt.setp(ax1.yaxis.get_offset_text(), fontsize='small')

ax2.xaxis.set_tick_params(labelsize='small')

ax1.tick_params(axis='x', which="major", width=1)
ax1.tick_params(axis='y', which="major", width=1, color=Vik_7.mpl_colors[1])
ax2.tick_params(axis='x', which="major", width=1)
ax2.tick_params(axis='y', which="major", width=1, color=Vik_7.mpl_colors[5])
ax1.spines['left'].set_linewidth(1)
ax1.spines['right'].set_linewidth(1)
ax1.spines['top'].set_linewidth(1)
ax1.spines['bottom'].set_linewidth(1)
ax2.spines['left'].set_linewidth(1)
ax2.spines['right'].set_linewidth(1)
ax2.spines['top'].set_linewidth(1)
ax2.spines['bottom'].set_linewidth(1)
ax2.set_xticklabels(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])

handles=[a.lines[0] for a in [ax1, ax2]]
labels = [
    "\\textbf{ HV }",
    "\\textbf{ TC }",
]
plt.legend(title='',
           handles=handles,
           labels=labels,
           fontsize='small',
           handlelength=3,
           handletextpad=1,
           markerscale=1,
           labelspacing=0.5,
           loc="lower right",
           bbox_to_anchor=(0.95, 0))

# plt.savefig("figures/parameter-sampling.pdf", dpi=300)


In [ ]:
results_list2 = ["results-database-20220126085808.json"]

from tinydb import TinyDB, Query

dataset2 = pd.DataFrame()

for results in results_list2:
    database = TinyDB("results/" + results)
    q = Query()
    temp = pd.DataFrame(database.all())
    dataset2 = pd.concat([dataset2, temp])
dataset2 = dataset2[[
    "run", "parts", "system_accuracies", "points", "constraints", "experiment_time"
]]
dataset2 = dataset2.fillna(0)
dataset2[['run', 'parts']] = dataset2[['run', 'parts']].astype(int)
refs2 = pd.DataFrame()
refs2['accuracy'] = dataset2['constraints'].map(lambda x: x['accuracy'][0])
refs2['number'] = dataset2['constraints'].map(lambda x: x['number'][0])
refs2['bandwidth'] = dataset2['constraints'].map(lambda x: x['bandwidth'][0])
ref2 = refs2.min().tolist()
dataset2 = dataset2.drop(columns=["constraints"])

hv2 = get_performance_indicator("hv", ref_point=np.array(ref2) * (-1))


In [ ]:
time_results2 = dataset2[['parts', 'run', 'experiment_time']]
time_results2 = time_results2.drop(columns=["run"])
time_results2 = time_results2.rename(columns={"experiment_time": "Time"})
time_results2

In [ ]:
df2=dataset2.copy(deep=True)
heuristic_dataset2=[]
if len(df2.index)>0:
    df2 = df2[['points', 'run', 'parts']]
    df2=list(df2.to_dict('records'))

    for item in df2:
        item['policy']="P"+str(item['parts'])
        points=item.pop('points')
        for point in points:
            point.update(item)
            heuristic_dataset2.append(point)

df2=pd.DataFrame(heuristic_dataset2).drop_duplicates(subset=None, keep='first', inplace=False)
# df.head()
hv_results2 = []
for parts, run in product(df2['parts'].unique(), df2['run'].unique()):
    filter1 = df2['parts'] == parts
    filter2 = df2['run'] == run
    temp = df2[['served', 'average_accuracy', 'bandwidth_reserved', 'parts',
               'run']].where(filter1 & filter2).dropna(axis=0, how='any')
    if temp.size > 0:
        data = temp[['average_accuracy', 'served', 'bandwidth_reserved']]
        result = {'parts': parts, 'run': run}
        front = data.values
        result.update({"HV": hv2.do(-front)})
        hv_results2.append(result)
hv_results2 = pd.DataFrame(hv_results2)
hv_results2

In [ ]:
plt.rcParams["axes.formatter.useoffset"] = True
from palettable.scientific.diverging import Vik_7

fig, ax1 = plt.subplots(figsize=(4, 2))

hv_results2[['parts']] = hv_results2[['parts']].astype("str")
time_results2[['parts']] = time_results2[['parts']].astype("str")

sns.lineplot(x='parts',
             y='HV',
             data=hv_results2,
             marker='o',
             markeredgecolor=None,
             markeredgewidth=None,
             markersize=4,
             linestyle='-',
             color=Vik_7.mpl_colors[1],
             ax=ax1)

ax2 = ax1.twinx()
ax2 = sns.lineplot(x='parts',
                   y='Time',
                   data=time_results2,
                   color=Vik_7.mpl_colors[5],
                   ax=ax2,
                   marker='X',
                   markeredgecolor=None,
                   markeredgewidth=None,
                   markersize=4,
                   linestyle='-')

ax1.xaxis.label.set_text('\\textbf{\# of Partitions ($\\times 10$)}')
ax2.yaxis.label.set_text('\\textbf{ TC (sec.)}')
ax1.yaxis.label.set_text('\\textbf{HV}')

ax1.yaxis.set_tick_params(labelsize='small')
ax2.yaxis.set_tick_params(labelsize='small')
plt.setp(ax1.yaxis.get_offset_text(), fontsize='small')
ax1.set_ylim(1045,1080)
ax2.set_ylim(100,150)
ax2.xaxis.set_tick_params(labelsize='small')

ax1.tick_params(axis='x', which="major", width=1)
ax1.tick_params(axis='y', which="major", width=1, color=Vik_7.mpl_colors[1])
ax2.tick_params(axis='x', which="major", width=1)
ax2.tick_params(axis='y', which="major", width=1, color=Vik_7.mpl_colors[5])
ax1.spines['left'].set_linewidth(1)
ax1.spines['right'].set_linewidth(1)
ax1.spines['top'].set_linewidth(1)
ax1.spines['bottom'].set_linewidth(1)
ax2.spines['left'].set_linewidth(1)
ax2.spines['right'].set_linewidth(1)
ax2.spines['top'].set_linewidth(1)
ax2.spines['bottom'].set_linewidth(1)
ax2.set_xticklabels(['-','1','2','3','4','5','6','7','8','9','10'])


handles=[a.lines[0] for a in [ax1, ax2]]
labels = [
    "\\textbf{ HV }",
    "\\textbf{ TC }",
]
plt.legend(
    title='',
    handles=handles,
    labels=labels,
    fontsize='small',
    handlelength=3,
    handletextpad=1,
    markerscale=1,
    labelspacing=0.5,
    loc="lower right",
    bbox_to_anchor=(0.95, 0)
)

# plt.savefig("figures/parameter-parts.pdf", dpi=300)


In [ ]:
fig, axes = plt.subplots(1,
                         2,
                         gridspec_kw={
                             'width_ratios': [1, 1],
                             "wspace": 0.25,
                         },
                         sharex=False,
                         sharey=False,
                         figsize=(10, 2))

plt.rcParams["axes.formatter.useoffset"] = True
from palettable.scientific.diverging import Vik_7

hv_results1[['s']] = hv_results1[['s']].astype("str")
time_results1[['s']] = time_results1[['s']].astype("str")

ax1=sns.lineplot(x='s',
             y='HV',
             data=hv_results1,
             marker='o',
             markeredgecolor=None,
             markeredgewidth=None,
             markersize=4,
             linestyle='-',
             color=Vik_7.mpl_colors[1],
             ax=axes[0])

ax2 = ax1.twinx()
ax2 = sns.lineplot(x='s',
                   y='Time',
                   data=time_results1,
                   color=Vik_7.mpl_colors[5],
                   ax=ax2,
                   marker='X',
                   markeredgecolor=None,
                   markeredgewidth=None,
                   markersize=4,
                   linestyle='-')

ax1.xaxis.label.set_text('\\textbf{\# of Weight Vectors ($\\times 1000$)}')
ax2.yaxis.label.set_text('\\textbf{ Time Cost (sec.)}')
ax1.yaxis.label.set_text('\\textbf{ Hypervolume }')
ax1.set_ylim(990, 1080)
ax1.yaxis.set_tick_params(labelsize='small')
ax2.yaxis.set_tick_params(labelsize='small')
plt.setp(ax1.yaxis.get_offset_text(), fontsize='small')

ax2.xaxis.set_tick_params(labelsize='small')

ax1.tick_params(axis='x', which="major", width=1)
ax1.tick_params(axis='y', which="major", width=1, color=Vik_7.mpl_colors[1])
ax2.tick_params(axis='x', which="major", width=1)
ax2.tick_params(axis='y', which="major", width=1, color=Vik_7.mpl_colors[5])
ax1.spines['left'].set_linewidth(1)
ax1.spines['right'].set_linewidth(1)
ax1.spines['top'].set_linewidth(1)
ax1.spines['bottom'].set_linewidth(1)
ax2.spines['left'].set_linewidth(1)
ax2.spines['right'].set_linewidth(1)
ax2.spines['top'].set_linewidth(1)
ax2.spines['bottom'].set_linewidth(1)
ax2.set_xticklabels(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])

handles1 = [a.lines[0] for a in [ax1, ax2]]
labels1 = [
    "\\textbf{ Hypervolume }",
    "\\textbf{ Time Cost }",
]
plt.legend(title='',
           handles=handles1,
           labels=labels1,
           fontsize='small',
           handlelength=3,
           handletextpad=1,
           markerscale=1,
           labelspacing=0.5,
           loc="lower right",
           bbox_to_anchor=(0.95, 0))

hv_results2[['parts']] = hv_results2[['parts']].astype("str")
time_results2[['parts']] = time_results2[['parts']].astype("str")

ax3=sns.lineplot(x='parts',
             y='HV',
             data=hv_results2,
             marker='o',
             markeredgecolor=None,
             markeredgewidth=None,
             markersize=4,
             linestyle='-',
             color=Vik_7.mpl_colors[1],
             ax=axes[1])

ax4 = ax3.twinx()
ax4 = sns.lineplot(x='parts',
                   y='Time',
                   data=time_results2,
                   color=Vik_7.mpl_colors[5],
                   ax=ax4,
                   marker='X',
                   markeredgecolor=None,
                   markeredgewidth=None,
                   markersize=4,
                   linestyle='-')

ax3.xaxis.label.set_text('\\textbf{\# of Partitions ($\\times 10$)}')
ax4.yaxis.label.set_text('\\textbf{ Time Cost (sec.)}')
ax3.yaxis.label.set_text('\\textbf{ Hypervolume }')

ax3.yaxis.set_tick_params(labelsize='small')
ax4.yaxis.set_tick_params(labelsize='small')
plt.setp(ax3.yaxis.get_offset_text(), fontsize='small')
ax3.set_ylim(1045, 1080)
ax4.set_ylim(100, 150)
ax4.xaxis.set_tick_params(labelsize='small')

ax3.tick_params(axis='x', which="major", width=1)
ax3.tick_params(axis='y', which="major", width=1, color=Vik_7.mpl_colors[1])
ax4.tick_params(axis='x', which="major", width=1)
ax4.tick_params(axis='y', which="major", width=1, color=Vik_7.mpl_colors[5])
ax3.spines['left'].set_linewidth(1)
ax3.spines['right'].set_linewidth(1)
ax3.spines['top'].set_linewidth(1)
ax3.spines['bottom'].set_linewidth(1)
ax4.spines['left'].set_linewidth(1)
ax4.spines['right'].set_linewidth(1)
ax4.spines['top'].set_linewidth(1)
ax4.spines['bottom'].set_linewidth(1)
ax4.set_xticklabels(['-', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])

handles2 = [a.lines[0] for a in [ax3, ax4]]
labels2 = [
    "\\textbf{ Hypervolume }",
    "\\textbf{ Time Cost }",
]
plt.legend(title='',
           handles=handles2,
           labels=labels2,
           fontsize='small',
           handlelength=3,
           handletextpad=1,
           markerscale=1,
           labelspacing=0.5,
           loc="lower right",
           bbox_to_anchor=(0.95, 0))

plt.savefig("figures/parameter.pdf", dpi=300)
